In [1]:
import random
import numpy as np
from deap import base, creator, tools, algorithms, gp
import matplotlib.pyplot as plt

In [2]:
class Train:
    def __init__(self, wagons, op):
        self.wagons = wagons
        self.op = op

    def __str__(self):
        return "Número de vagones:" + str(self.wagons) + "\n" + "Muelle de operaciones:" + str(self.op)

def random_trains_generation(n):
    train_list = []
    for _ in range(n):
        wagons = random.randint(10, 30)  # Cada tren puede arrastrar entre 10 y 30 vagones
        op = random.choice(["op1", "op2", "op3"])  # A cada tren se le asigna un tipo de carga
        train_list.append(Train(wagons, op))
    return train_list


incoming_trains = random_trains_generation(25)  # supongamos que entran 7 trenes


# Crea una nueva reordenación de los trenes entrantes
def create_queue():
    aux = incoming_trains[:]
    random.shuffle(aux)
    return aux


class Trains(list):
    def __init__(self):
        list.extend(self, create_queue())

def evaluation(individual):
    dock_op_counters = {"op1" : 0, "op2" : 0, "op3" : 0}
    time = 0

    for train in individual:
        next_delay = train.wagons
        if dock_op_counters[train.op] == 0:
            dock_op_counters[train.op] = next_delay
        else:
            current_delay = dock_op_counters[train.op]
            for k, v in dock_op_counters.items():
                dock_op_counters[k] = max(0, v - current_delay)
            dock_op_counters[train.op] = next_delay
            time += current_delay

    time += max(dock_op_counters.values())
    return time,

def mutate(train_list):
    index_1, index_2 = np.random.choice(len(train_list), 2, replace=False)
    list_index_2 = train_list[index_2]
    train_list[index_2] = train_list[index_1]
    train_list[index_1] = list_index_2
    return train_list,

def crossover(list_1, list_2):
    list_a = []

    resultado = []

    index = np.random.choice(len(list_1))

    for _ in range(2):

        list_a_set = set(list_a)

        list_a_faltantes = []

        for a in list_2:
            if a not in list_a_set:
                list_a_faltantes.append(a)

        for a in list_2[index:]:
            if a in list_1[0:index]:
                list_2[list_2.index(a)] = list_a_faltantes[0]
                list_a_faltantes = np.delete(list_a_faltantes, 0)

        resultado.append(list_1[0:index] + list_2[index:])

        variable_temporal = list_1

        list_1 = list_2
        list_2 = variable_temporal

    return resultado[0], resultado[1]

In [3]:
##################### DEAP

creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", Trains, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("individual", creator.Individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluation)

toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.01)
toolbox.register("crossover", tools.cxOrdered)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxOrdered)

pop = toolbox.population(n=10)

In [4]:
hof = tools.HallOfFame(4)

stats = tools.Statistics(lambda indiv: indiv.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg  	min	max
0  	10    	273.8	255	324
1  	3     	265.8	258	278


TypeError: list indices must be integers or slices, not Train

In [ ]:
print("Best individual is: %s\n with fitness: %s" % (hof[0], hof[0].fitness))

gen, avg, min_, max_ = logbook.select("gen", "avg", "min", "max")
plt.plot(gen, avg, label="average")
plt.plot(gen, min_, label="minimum")
plt.plot(gen, max_, label="maximum")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="lower right")
plt.show()

In [ ]:
for i in pop[0]:
    print(i)

In [ ]:
for i in incoming_trains:
    print(i)

In [ ]:
cont1 = 0
cont2 = 0
cont3 = 0
for i in incoming_trains:
    if i.op == "op1":
        cont1 += i.wagons

    if i.op == "op2":
        cont2 += i.wagons

    if i.op == "op3":
        cont3 += i.wagons

print(max(cont1, cont2, cont3))

print(cont1, cont2, cont3)

In [ ]:
print(len(incoming_trains))